In [14]:
import json
from PIL import Image, ImageDraw
import io
import math
import numpy as np
import os
import sys
import tensorflow as tf
import yaml

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
with open('./sample_data/labels.json') as json_file:
    labels = json.load(json_file)

In [7]:
def xys_to_bitmap(xys, height, width, rescale=1.0):
    bitmap = np.zeros((int(height*rescale), int(width*rescale), 1), dtype=np.float32)
    for x, y in xys:
        bitmap[int(y*rescale), int(x*rescale), 0] = 1.0
    return bitmap

In [8]:
def get_point_from_bbox(x,y,width,height):
    return (x+width/2,y+height/2)

In [12]:
def get_points(flabels, cl="human"):
    labels = []
    for o in flabels["Label"]["objects"]:
        if o["value"] != cl:
            continue
        bbox = o["bbox"]
        labels.append(tuple(get_point_from_bbox(bbox['left'], bbox['top'], bbox['width'], bbox['height'])))
    return labels

In [10]:
def bitmap_to_single_channel_pil_image(bitmap):
    h, w, c = bitmap.shape
    assert c == 1
    bitmap = np.uint8(bitmap[:,:,0] * 255)
    return Image.fromarray(bitmap, mode='L')

In [17]:
for i, frame in enumerate(labels):
    xys = get_points(frame, "human")
    bitmap = xys_to_bitmap(xys, 720, 1280)
    single_chan_img = bitmap_to_single_channel_pil_image(bitmap)
    single_chan_img.save("./sample_data/testing/"+frame["ID"]+".png")